# Chapter 5: Vector IR

Instead of going straight from Toy IR to RISC-V, we'll go through an intermediate dialect,
with some simpler types. In this IR, tensors represented as two 1D vectors, containing
the shape and data information. We can still do pointwise addition on the contents of the
vectors.

In [1]:
from compiler import (parse_toy, print_op, optimise_toy, lower_from_toy, 
                                  optimise_vir, lower_to_riscv, emulate_riscv)

example = """
def main() {
  var a<2, 3> = [[1, 2, 3], [4, 5, 6]];
  var b<6> = [1, 2, 3, 4, 5, 6];
  var c<2, 3> = b;
  var d = a + c;
  print(d);
}
"""

toy_0 = parse_toy(example)
toy_1 = optimise_toy(toy_0)
vir_0 = lower_from_toy(toy_1)

print_op(vir_0)

"builtin.module"() ({
  "toy.func"() ({
    %0 = "toy.vector.constant"() {"data" = [#int<2>, #int<3>], "label" = "tensor_shape"} : () -> #riscv_ssa.reg
    %1 = "toy.vector.constant"() {"data" = [#int<1>, #int<2>, #int<3>, #int<4>, #int<5>, #int<6>], "label" = "tensor_data"} : () -> #riscv_ssa.reg
    %2 = "toy.tensor.make"(%0, %1) : (#riscv_ssa.reg, #riscv_ssa.reg) -> #riscv_ssa.reg
    %3 = "toy.vector.constant"() {"data" = [#int<2>, #int<3>], "label" = "tensor_shape"} : () -> #riscv_ssa.reg
    %4 = "toy.vector.constant"() {"data" = [#int<1>, #int<2>, #int<3>, #int<4>, #int<5>, #int<6>], "label" = "tensor_data"} : () -> #riscv_ssa.reg
    %5 = "toy.tensor.make"(%3, %4) : (#riscv_ssa.reg, #riscv_ssa.reg) -> #riscv_ssa.reg
    %6 = "toy.tensor.shape"(%2) : (#riscv_ssa.reg) -> #riscv_ssa.reg
    %7 = "toy.tensor.data"(%2) : (#riscv_ssa.reg) -> #riscv_ssa.reg
    %8 = "toy.tensor.data"(%5) : (#riscv_ssa.reg) -> #riscv_ssa.reg
    %9 = "toy.vector.add"(%7, %8) : (#riscv_ssa.reg, #riscv_s

### Lowering Toy to Vector IR


TODO: lower toy.add to toy.tensor.add

### Optimising VIR

As we can see, the code expanded quite a bit. The vector constants are initialised
individually, and then combined using the `toy.tensor.make` operation. One optimisation we
can implement is to use the original data vector instead of using the one from the tensor:

``` python
# Before
shape = ...
data_0 = ...
tensor = Tensor(shape, data_0)
data_1 = tensor.data

# After
shape = ...
data_0 = ...
tensor = Tensor(shape, data_0)
data_1 = data_0
```

If the tensor is unused, we can delete it altogether. Let's see what that gives us.

In [2]:
from typing import cast

from xdsl.ir import OpResult, Operation
from xdsl.dialects.builtin import DenseIntOrFPElementsAttr
from xdsl.pattern_rewriter import (op_type_rewrite_pattern, RewritePattern,
                                   PatternRewriter, PatternRewriteWalker)

from toy.rewrites import RemoveUnusedOperations

import vector_ir.dialect as tvd

class SimplifyRedundantDataAccess(RewritePattern):

    @op_type_rewrite_pattern
    def match_and_rewrite(self, op: tvd.TensorDataOp,
                          rewriter: PatternRewriter):
        """
        Fold tensor(t_shape, t_data).data -> t_data
        """
        # Look at the input of the current transpose.
        tensor_data_input = op.rs1
        if not isinstance(tensor_data_input, OpResult):
            # Input was not produced by an operation, could be a function argument
            return

        tensor_make_op = tensor_data_input.op
        if not isinstance(tensor_make_op, tvd.TensorMakeOp):
            # Input defined by a constant passed in? If not, no match.
            return

        rewriter.replace_op(op, [], [tensor_make_op.rs2])

PatternRewriteWalker(SimplifyRedundantDataAccess()).rewrite_module(vir_0)
PatternRewriteWalker(RemoveUnusedOperations()).rewrite_module(vir_0)
print_op(vir_0)


"builtin.module"() ({
  "toy.func"() ({
    %0 = "toy.vector.constant"() {"data" = [#int<2>, #int<3>], "label" = "tensor_shape"} : () -> #riscv_ssa.reg
    %1 = "toy.vector.constant"() {"data" = [#int<1>, #int<2>, #int<3>, #int<4>, #int<5>, #int<6>], "label" = "tensor_data"} : () -> #riscv_ssa.reg
    %2 = "toy.tensor.make"(%0, %1) : (#riscv_ssa.reg, #riscv_ssa.reg) -> #riscv_ssa.reg
    %3 = "toy.vector.constant"() {"data" = [#int<2>, #int<3>], "label" = "tensor_shape"} : () -> #riscv_ssa.reg
    %4 = "toy.vector.constant"() {"data" = [#int<1>, #int<2>, #int<3>, #int<4>, #int<5>, #int<6>], "label" = "tensor_data"} : () -> #riscv_ssa.reg
    %5 = "toy.tensor.shape"(%2) : (#riscv_ssa.reg) -> #riscv_ssa.reg
    %6 = "toy.vector.add"(%1, %4) : (#riscv_ssa.reg, #riscv_ssa.reg) -> #riscv_ssa.reg
    %7 = "toy.tensor.make"(%5, %6) : (#riscv_ssa.reg, #riscv_ssa.reg) -> #riscv_ssa.reg
    "toy.print"(%7) : (#riscv_ssa.reg) -> ()
    "toy.return"() : () -> ()
  }) {"sym_name" = "main", "functio

In [3]:
from vector_ir.rewrites import SimplifyRedundantShapeAccess

PatternRewriteWalker(SimplifyRedundantShapeAccess()).rewrite_module(vir_0)
PatternRewriteWalker(RemoveUnusedOperations()).rewrite_module(vir_0)

print_op(vir_0)

"builtin.module"() ({
  "toy.func"() ({
    %0 = "toy.vector.constant"() {"data" = [#int<2>, #int<3>], "label" = "tensor_shape"} : () -> #riscv_ssa.reg
    %1 = "toy.vector.constant"() {"data" = [#int<1>, #int<2>, #int<3>, #int<4>, #int<5>, #int<6>], "label" = "tensor_data"} : () -> #riscv_ssa.reg
    %2 = "toy.vector.constant"() {"data" = [#int<1>, #int<2>, #int<3>, #int<4>, #int<5>, #int<6>], "label" = "tensor_data"} : () -> #riscv_ssa.reg
    %3 = "toy.vector.add"(%1, %2) : (#riscv_ssa.reg, #riscv_ssa.reg) -> #riscv_ssa.reg
    %4 = "toy.tensor.make"(%0, %3) : (#riscv_ssa.reg, #riscv_ssa.reg) -> #riscv_ssa.reg
    "toy.print"(%4) : (#riscv_ssa.reg) -> ()
    "toy.return"() : () -> ()
  }) {"sym_name" = "main", "function_type" = () -> ()} : () -> ()
}) : () -> ()
